In [1]:
from spinbox import *
from itertools import count
seeder = count(0,1) # generates a list of counting numbers for rng "seeds" (not really seeds in Numpy, but indices of seeds)

# The `spinbox` tutorial
## Part 3 : Basic imaginary-time propagation in the full basis space

This notebook will show how to approach imaginary-time propagation of `HilbertState` objects using a simple interaction.

Note that `spinbox` does not have any built-in kinetic operator stuff. We are focused on the interaction, although any operator can be made using a `HilbertOperator`.

Consider the following imaginary-time propagator amplitude between states $S$ and $S'$ of a 2-particle system with spin and isospin.

$$\langle S' | \exp \left[  - \frac{\delta\tau}{2} A^\sigma_{1x2x} \sigma_{1x} \sigma_{2x} \right] | S \rangle $$

First, instantiate some states.

In [9]:
s = HilbertState(2, ketwise=True, isospin=True).randomize(next(seeder))
sp = HilbertState(2, ketwise=False, isospin=True).randomize(next(seeder))
print(s,"\n",sp)

HilbertState ket of 2 particles:
[[ 0.17246905+0.2116545j ]
 [ 0.29093646+0.29458772j]
 [-0.41815329-0.00427012j]
 [-0.02259457+0.22661052j]
 [ 0.1660171 -0.14835017j]
 [ 0.22144063-0.13368808j]
 [ 0.1070711 +0.01331502j]
 [ 0.24518339+0.04609j   ]
 [ 0.04748643-0.26186675j]
 [ 0.09028117-0.28344116j]
 [ 0.02927192+0.05815128j]
 [-0.17586613-0.14120077j]
 [-0.13865265+0.19789296j]
 [ 0.06216461+0.06461494j]
 [-0.09501872+0.05030771j]
 [ 0.20824231+0.0341367j ]] 
 HilbertState bra of 2 particles:
[[ 2.32506725e-04-0.25406501j  5.64647864e-02-0.08649226j -5.18137796e-02-0.35934306j -1.68327461e-01-0.24373075j -8.59356393e-02-0.34809951j -1.87427439e-01-0.0444337j   1.13675193e-02-0.23955536j
   2.53309114e-01+0.05127067j -9.30301291e-02+0.02962694j -1.17273660e-01-0.03533112j  9.25832297e-02-0.47568342j  6.74538902e-02-0.10181635j  1.99239564e-02-0.00916698j -1.75864315e-01+0.02141611j
  -5.52877850e-03-0.28920529j  1.31416679e-01-0.09029838j]]


The coupling $A^\sigma_{1x2x}$ is some real number determined by other calculations, so we will just plug in some number. For more complicated situations, we should use a `TwoBodyCoupling` for this.

We can compute the amplitude "exactly" using the Pade approximant exponentiation, like so.

In [10]:
coupling = 3.14
dt = 0.01
force = HilbertOperator(2).apply_sigma(0,0).apply_sigma(1,0).scale(coupling)  # note that isospin is True by default
propagator = force.scale(- 0.5j * dt).exp()
amplitude_exact = sp * propagator * s
print("exact = ", amplitude_exact)

exact =  [[0.13860788+0.09865043j]]


Next, lets compute this again using a Hubbard-Stratonovich transform, using the `HilbertPropagatorHS` class. 

The HS and RBM propagator classes all have built-in methods for the usual forces, as well as methods `onebody()` and `twobody_sample()` which apply one-body propagators and samples of the "two-body" summand for the appropriate method.

Note that here we are "balancing" the distribution: for each sample of the Gaussian auxiliary field $x$ we also evaluate at $-x$.

In [13]:
n_samples = 100000
prop_hs = HilbertPropagatorHS(2, dt)
gaussian_fields = np.random.standard_normal(size=n_samples)
operator_i = HilbertOperator(2).apply_sigma(0,0)
operator_j = HilbertOperator(2).apply_sigma(1,0)
amplitude_distribution_hs = np.zeros(2*n_samples, dtype=complex)
for ix,x in enumerate(gaussian_fields):
    amplitude_distribution_hs[2*ix:2*(ix+1)]= sp * prop_hs.twobody_sample(coupling, x, operator_i, operator_j) * s
    x_flip = -x
    amplitude_distribution_hs[2*ix+1:2*(ix+1)+1]= sp * prop_hs.twobody_sample(coupling, x_flip, operator_i, operator_j) * s
amplitude_hs = np.mean(amplitude_distribution_hs)
amplitude_hs_unc = np.std(amplitude_distribution_hs)/np.sqrt(n_samples)
print(f"HS = {amplitude_hs} +/- {amplitude_hs_unc}")
print(f"relative error = {(amplitude_exact - amplitude_hs)/amplitude_exact}")

HS = (0.13863316135607667+0.09865983951096717j) +/- 6.95304805172661e-05
relative error = [[-0.00015315+4.11303512e-05j]]


That's in agreement with the exact. Finally, let's use `HilbertPropagatorRBM` to do the same. We also balance this calculation: for each sample of the binary auxiliary field $h$ we also evaluate at $1-h$.

In [14]:
prop_rbm = HilbertPropagatorRBM(2, dt)
binary_fields = np.random.randint(0,2,size=n_samples)
amplitude_distribution_rbm = np.zeros(2*n_samples, dtype=complex)
for ih,h in enumerate(binary_fields):
    amplitude_distribution_rbm[2*ih:2*(ih+1)] = sp * prop_rbm.twobody_sample(coupling, h, operator_i, operator_j) * s
    h_flip = 1-h
    amplitude_distribution_rbm[2*ih+1:2*(ih+1)+1] = sp * prop_rbm.twobody_sample(coupling, h_flip, operator_i, operator_j) * s
amplitude_rbm = np.mean(amplitude_distribution_rbm)
amplitude_rbm_unc = np.std(amplitude_distribution_rbm)/np.sqrt(n_samples)
print(f"RBM = {amplitude_rbm} +/- {amplitude_rbm_unc}")
print(f"relative error = {(amplitude_exact - amplitude_rbm)/amplitude_exact}")

RBM = (0.13860787609549666+0.09865043223241828j) +/- 8.757946600012966e-05
relative error = [[8.5616613e-17-1.61057974e-16j]]


That looks good. While we are here, let's try a three-body force too. We can do the same thing but compute this propagator amplitude for a 3-particle system. We do not have a Hubbard-Stratonovich transformation for 3-body forces, but we can use the RBM.

$$\langle S' | \exp \left[  - \frac{\delta\tau}{2} A^\sigma_{1x2x3x} \sigma_{1x} \sigma_{2x} \sigma_{3x} \right] | S \rangle $$

Note that the scaling for three-body is worse, so depending on `n_samples` this may take a while. Later, we will look at how to do general calculations using the `Integrator` class, which has thread-parallelization.

In [7]:
s = HilbertState(3, ketwise=True, isospin=True).randomize(next(seeder))
sp = HilbertState(3, ketwise=False, isospin=True).randomize(next(seeder))

coupling = 3.14
dt = 0.01
force = HilbertOperator(3).apply_sigma(0,0).apply_sigma(1,0).apply_sigma(2,0).scale(coupling) 
propagator = force.scale(- 0.5j * dt).exp()
amplitude_exact = sp * propagator * s
print("exact = ", amplitude_exact)

n_samples = 1000
prop_rbm = HilbertPropagatorRBM(3, dt)
operator_i = HilbertOperator(3).apply_sigma(0,0)
operator_j = HilbertOperator(3).apply_sigma(1,0)
operator_k = HilbertOperator(3).apply_sigma(2,0)
binary_fields = np.random.randint(0,2,size=(n_samples,4))
amplitude_distribution_rbm = []
for h in tqdm(binary_fields):
    amplitude_distribution_rbm.append( sp * prop_rbm.threebody_sample(coupling, h, operator_i, operator_j, operator_k) * s)
    h_flip = 1 - h
    amplitude_distribution_rbm.append( sp * prop_rbm.threebody_sample(coupling, h_flip, operator_i, operator_j, operator_k) * s)
amplitude_rbm = np.mean(amplitude_distribution_rbm)
amplitude_rbm_unc = np.std(amplitude_distribution_rbm)/np.sqrt(n_samples)
print(f"RBM = {amplitude_rbm} +/- {amplitude_rbm_unc}")
print(f"error = {(amplitude_exact - amplitude_rbm)}")
print(f"relative error = {(amplitude_exact - amplitude_rbm)/amplitude_exact}")

exact =  [[-0.00344316+0.0056979j]]


100%|██████████| 10000/10000 [00:50<00:00, 196.45it/s]

RBM = (-0.0029961029297486032+0.0065467270985524965j) +/- 0.0011992195190054615
error = [[-0.00044706-0.00084883j]]
relative error = [[-0.07439407+0.12341591j]]
